# NRC-09 Darks: Noise Stability & Dark Current Performance

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Analyze the dark current exposures](#dark)
* [Systematics from cosmic rays](#jump)
* [Effective noise](#noise)

<a id='objective'></a>
## Objective

These will be the first darks taken at the nominal operating temperature. We will measure the dark current in the long-wave detectors and assess stability of noise and transients for all full-frame and subarray observations. We will need to check exposures for all configurations (DETECTOR, CORONMSK, NOUPUTS). Compare with ground test data. Generate new dark reference files.

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. For this analysis, you will also need the dark current tools in the jwst_reffiles package, which is used to generate dark current reference files. 

Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used that are not included in the ```jwst``` package:
```python
pip install ipython jupyter matplotlib pylint pandas
```

and install [jwst_reffiles](https://jwst-reffiles.readthedocs.io/en/latest/official_bad_pixel_mask.html?highlight=hot%20pixel):
```python
pip install git+https://github.com/spacetelescope/jwst_reffiles.git
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
from jwst.datamodels import dqflags
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from jwst_reffiles.dark_current import dark_reffile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator
%matplotlib inline

In [ ]:
def plot_jump(signal, jump_group, xpixel=None, ypixel=None, slope=None):
    """Hilbert's function to plot the signal up the ramp for a
    pixel and show the location of flagged jumps.
    
    Parameters
    ----------
    signal : numpy.ndarray
        1D array of signal values
        
    jump_group : list
        List of boolean values whether a jump is present or
        not in each group
        
    slope : numpy.ndarray
        1D array of signal values constructed from the slope
    """
    groups = np.arange(len(signal))
    fig = plt.figure(figsize=(6, 6))
    ax = plt.subplot()

    plt.plot(groups, signal, marker='o', color='black')
    plt.plot(groups[jump_group], signal[jump_group], marker='o', color='red',
             label='Flagged Jump')
    
    if slope is not None:
        plt.plot(groups, slope, marker='o', color='blue', label='Data from slope')
        
    plt.legend(loc=2)

    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()
    plt.subplots_adjust(top=0.95)
    
    if xpixel and ypixel:
        plt.title('Pixel ('+str(xpixel)+','+str(ypixel)+')')

In [ ]:
def plot_jumps(signals, jump_groups, pixel_loc, slopes=None):
    """Hilbert's function to plot the ramp and show the jump location
    for several pixels. For simplicity, let's force the input
    number of pixels to be a square. 
    
    Parameters
    ----------
    signals : numpy.ndarray
        2D array (groups x pix) of signal values
        
    jump_groups : numpy.ndarray
        2D array containing boolean entries for each group of
        each pixel, describing where the jumps were found
        
    pixel_loc : list
        List of 2-tuples containing the (x, y)
        location of the pixels with the jumps
        
    slopes : numpy.ndarray
        2D array (groups x pix) of linear signal values
        If not None, these will be overplotted onto the
        plots of signals
    """
    num_group, num_pix = signals.shape
    root = np.sqrt(num_pix)
    if int(root + 0.5) ** 2 != num_pix:
        raise ValueError('Number of pixels input should be a square.')
    
    root = int(root)
    groups = np.arange(num_group)
    fig, axs = plt.subplots(root, root, figsize=(10, 10))

    for index in range(len(pixel_loc)):
        i = int(index % root)
        j = int(index / root)
        axs[i, j].plot(groups, signals[:, index], marker='o', color='black')
        j_grp = jump_groups[:, index]
        axs[i, j].plot(groups[j_grp], signals[j_grp, index],
                       marker='o', color='red')
        
        if slopes is not None:
            axs[i, j].plot(groups, slopes[:, index], marker='o', color='blue')
        
        axs[i, j].set_title('Pixel ({}, {})'.format(pixel_loc[index][1], pixel_loc[index][0]))
        
    plt.xlabel('Groups')
    plt.ylabel('Signal (DN)')
    fig.tight_layout()

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

This analysis relies a list of exposures that have been through the following pipeline steps:

* dq_init
* saturation
* superbias
* refpix
* linearity
* jump 
* persistence

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
dark_files = sorted(glob(base_dir+'*_dark.fits'))

In [ ]:
# files = glob(base_dir+ground+'/NRC*_484_*uncal.fits')
# files = glob("*linearitystep.fits")
files = glob("/Users/acanipe/jwebbinars/data_files/*jump*fits")
files

<a id='dark'></a>
## Analyze the dark current exposures

Expected dark current values based on ground data are shown in the table below. 

|__SW dark current (1000s)__         | __LW dark current (1000s)__   |
|:---                   |:---              |
__ 1.9 ± 1.1 e–         | 27 ± 5 e–        |



The dark current function creates a dark current reference file in the proper format. Details are in [the documentation](https://jwst-reffiles.readthedocs.io/en/latest/api.html?highlight=dark%20current#module-jwst_reffiles.dark_current.dark_reffile).

The steps are:

1. Get file headers from each file. Check for consistency. Extract CR flags from the JUMP version of the files.

2. From these, create arrays giving the group number of the first cosmic ray hit in each pixel for each integration.

3. Create images of how many dark current measurements are considered good for each pixel and each group. All groups after the first CR hit in a pixel are considered bad and not used.

4. Read in the data. To avoid memory problems only read in N groups at a time, where N is determined from user inputs.

5. Flag all groups that occur between the first CR hit and the end of the integration as bad.

6. From the remaining good data, calculate the sigma-clipped mean dark signal in each pixel for each group (i.e. take the mean across integrations). Also calculate the sigma-clipped stdard deviation.

7. Save the sigma-clipped mean as the dark data, and the sigma-clipped stdev as the uncertainty in a DarkModel instance. The DQ array is currently all zeros, anticipating bad pixels being placed in the bad pixel reference file.

In [ ]:
output_file = analysis_dir+"test_dark.fits"

In [ ]:
dark_reffile(file_list=files, max_equivalent_groups=60, sigma_threshold=3, 
             pedigree=None, descrip=None, author=None, use_after=None, history=None, 
             output_file=output_file)

<a id='jump'></a>
## Systematics from cosmic rays

For now, use the JWST pipeline jump detection algorithm and jump scripts from Hilbert, and eventually we will suplement this with Misselt's analysis. The jump step should have been run on the input data for the anlaysis above (our input dark current exposures). More information on the jump detection algorithm is in [the documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/index.html). 

Start by finding which pixels have been flagged with jumps. For our tests we will choose on of the input files. 

In [ ]:
test_case = files[0]
jump = datamodels.RampModel(test_case)

In [ ]:
# Find total number of jump flags added (including some pixels
# that have multiple groups flagged).
jump_flags = np.where(jump.groupdq & dqflags.pixel['JUMP_DET'] > 0)
print('{} jump flags detected.'.format(len(jump_flags[0])))

In [ ]:
# Create 4D map of the jump flags and collapse to 2D map for the number of jumps per pixel 
jump_map = (jump.groupdq & dqflags.pixel['JUMP_DET'] > 0)
jump_map_2d = np.sum(jump_map[0, :, :, :], axis=0)

In [ ]:
# Find how many pixels have jump flags
jump_map_indexes = np.where(jump_map_2d > 0)
impacted_pix = np.sum(jump_map_2d > 0)
total_pix = jump.meta.subarray.ysize * jump.meta.subarray.xsize
print(('{} pixels ({:.2f}% of the detector) have been flagged with '
      'at least one jump.'.format(impacted_pix, 100. * impacted_pix / total_pix)))

Look at some of the pixels. 

In [ ]:
# Create an array of groups for the x-axis 
group_indexes = np.arange(jump_map.shape[1]).astype(int)

In [ ]:
# Pick one pixel with a flagged jump, and find the group(s) with the jump flags
j_index = 500
jumpy = jump_map_indexes[0][j_index]
jumpx = jump_map_indexes[1][j_index]
jump_grp = jump_map[0, :, jumpy, jumpx]
print('Jump located in group(s) {} of pixel ({}, {})'.format(group_indexes[jump_grp], jumpx, jumpy))

In [ ]:
# Plot the ramp for this pixel
plot_jump(jump.data[0, :, jumpy, jumpx], jump_grp, xpixel=jumpx, ypixel=jumpy)

Plot a grid of some examples of flagged jumps. The red points are the groups with flagged jumps. These groups will be ignored in subsequent ramp-fitting.

In [ ]:
indexes_to_plot = [200, 401, 600, 30010, 31000, 1202, 1400, 21600, 10112]
jump_data = np.zeros((jump.shape[1], len(indexes_to_plot)))
jump_grps = np.zeros((jump.shape[1], len(indexes_to_plot))).astype(bool)
jump_locs = []
for counter, idx in enumerate(indexes_to_plot):
    #integ, grp, y, x = jump_flags[idx]
    y = jump_map_indexes[0][idx]
    x = jump_map_indexes[1][idx]
    grp = jump_map[0, :, y, x]

    jump_data[:, counter] = jump.data[0, :, y, x]
    jump_grps[:, counter] = grp
    jump_locs.append((x, y))

In [ ]:
plot_jumps(jump_data, jump_grps, jump_locs)

<a id='noise'></a>
## Effective noise

For this analysis, you need at least 10 total ramps to get effective noise (note that you can still do spatial effective noise to see how it falls off up the ramps). 